In [1]:
!pip install requests
!pip install pandas
!pip install beautifulsoup4

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
url = 'https://www.regelleistung.net/ext/data/?lang=en'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 '
                  'Safari/537.36',
}

In [4]:

def collect_data(tso_id_, data_type_):
    request_data = {
        'from': '05.02.2022',
        '_download': 'on',
        'tsoId': tso_id_,
        'dataType': data_type_,
    }

    column_list = []
    final_list = []

    response = None
    try:
        response = requests.post(url, data=request_data, headers=headers)
    except Exception:
        print('Unable to connect with new connection')

    if response is not None and response.status_code == 200:

        response_text = response.content

        soup = BeautifulSoup(response_text, "html.parser")

        # Select Table Element
        table_ele = soup.find('table', {'id': 'data-table'})

        # Find all column
        columns = table_ele.find_all('th')

        for column in columns:
            column_list.append(column.text)

        # Find all rows
        rows = table_ele.find_all('tr')
        for row in rows:
            temp = []
            all_tds = row.find_all('td')

            if len(all_tds) == 0:
                continue

            for cell in all_tds:
                cell_content = cell.text
                temp.append(cell_content)

            final_list.append(temp)

    return pd.DataFrame(final_list, columns=column_list)

In [5]:

tso_ids = {
    '3': 'Amprion',
    '2': 'TenneT',
    '1': 'TransnetBW',
    '6': 'Netzregelverbund',
    '11': 'IGCC',
    '-42': 'Netzregelverbund detailliert',
}
data_types = {
    'MRL': 'MR',
    'SRL': 'SCR',
    'RZ_SALDO': 'RZ_SALDO',
    'REBAP': 'REBAP',
    'ZUSATZMASSNAHMEN': 'emergency power',
    'NOTHILFE': 'emergency assistance',
}

frames = []

for tso_id in tso_ids.items():
    tso_id_key, tso_id_value = tso_id

    for data_type in data_types.items():
        data_type_key, data_type_value = data_type

        temp_df = collect_data(tso_id_key, data_type_key)
        temp_df['tso_id'] = tso_id_value
        temp_df['data_type'] = data_type_value
        frames.append(temp_df)


In [6]:
df_concat = pd.concat(frames)
df_concat

,Date,Time from,Time to,betr. NEG [MW],betr. POS [MW],qual. NEG [MW],qual. POS [MW],tso_id,data_type,betr. [MW],qual. [MW],qual. [EUR/MWh],NRV balance more than 80% of the contracted control reserve,MOL Deviation,Explanation
0,05.02.2022,00:00,00:15,"0,000","0,000",-,-,Amprion,MR,NaN,NaN,NaN,NaN,NaN,NaN
1,05.02.2022,00:15,00:30,"0,000","0,000",-,-,Amprion,MR,NaN,NaN,NaN,NaN,NaN,NaN
2,05.02.2022,00:30,00:45,"0,000","0,000",-,-,Amprion,MR,NaN,NaN,NaN,NaN,NaN,NaN
3,05.02.2022,00:45,01:00,"0,000","0,000",-,-,Amprion,MR,NaN,NaN,NaN,NaN,NaN,NaN
4,05.02.2022,01:00,01:15,"0,000","0,000",-,-,Amprion,MR,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,05.02.2022,19:00,19:15,NaN,NaN,NaN,NaN,Netzregelverbund detailliert,REBAP,NaN,NaN,-,NaN,NaN,NaN
77,05.02.2022,19:15,19:30,NaN,NaN,NaN,NaN,Netzregelverbund detailliert,REBAP,NaN,NaN,-,NaN,NaN,NaN
78,05.02.2022,19:30,19:45,NaN,NaN,NaN,NaN,Netzregelverbund detailliert,REBAP,NaN,NaN,-,NaN,NaN,NaN
79,05.02.2022,19:45,20:00,NaN,NaN,NaN,NaN,Netzregelverbund detailliert,REBAP,NaN,NaN,-,NaN,NaN,NaN


In [7]:
df_concat.to_csv('data.csv')